#load origin data
import os
import numpy as np

lines = np.loadtxt('./sample_submission.csv',delimiter=',', dtype=str)
data = lines[1:].astype(int)

lines = np.loadtxt('./train.csv', delimiter=',', dtype=str)
[nrow, ncol] = lines.shape
print nrow, ncol
#lines[0]
train_id = lines[1:, 0].astype(int)
train_target = lines[1:, ncol-1].astype(int)
train_data_ori = lines[1:, 1:ncol-1].astype(float)
feature_names = lines[0,:]

#read test data
lines = np.loadtxt('./test.csv', delimiter=',', dtype=str)
test_id = lines[1:, 0].astype(int)
test_data_ori = lines[1:, 1:ncol-1].astype(float)
print 'Done load train&test data'

#delete single-value features

from sklearn.feature_selection import VarianceThreshold

varThresProb = 1.
sel = VarianceThreshold(varThresProb*(1-varThresProb))
train_data = sel.fit_transform(train_data_ori)

#MODEL PARAM
param_selected_indices = sel.get_support(indices=True)

print 'Done delete redundant features'

In [2]:
#load data
import numpy as np
import pandas as pd

df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')

In [3]:
#load data and delete redundant data

remove = []

for col in df_train:
    if df_train[col].std() == 0:
        remove.append(col)

df_train.drop(remove, axis=1, inplace=True)
df_test.drop(remove, axis=1, inplace=True)
print (df_train.values).shape

remove = []
for i in range(df_train.shape[1]):
    for j in range(i+1, df_train.shape[1]):
        if np.array_equal(df_train[df_train.columns[i]].values,df_train[df_train.columns[j]].values):
            remove.append(df_train.columns[j])
df_train.drop(remove, axis=1, inplace=True)
df_test.drop(remove, axis=1, inplace=True)

print (df_train.values).shape

train_data_id = df_train['ID'].values
test_data_id = df_test['ID'].values
#print train_data_id.shape

train_data_target = df_train['TARGET'].values
#print train_data_target.shape

train_data = df_train.drop(['ID', 'TARGET'],axis=1).values
test_data = df_test.drop(['ID'],axis=1).values
print train_data.shape
print test_data.shape

(76020, 337)
(76020, 308)
(76020, 306)
(75818, 306)


In [4]:
#training data type analyses

def type_analyses(nums):
    data_type = 1 #int
    for num in nums:
        if(round(float(num)) != float(num)): #float
            data_type = 2
    if (data_type == 1) & (nums.shape[0]==2):
        data_type = 0
    return data_type

unique_data = []
unique_data_size = []
unique_data_type = []
typeName = ['bool', 'int', 'float']

for i in range(train_data.shape[1]):
    values = np.unique(train_data[:,i])
    unique_data.append(values)
    unique_data_size.append(values.shape)
    feature_type = type_analyses(values)
    unique_data_type.append(feature_type)


param_bool_feature_indices = np.argwhere(np.array(unique_data_type)==0).reshape(unique_data_type.count(0))
param_int_feature_indices = np.argwhere(np.array(unique_data_type)==1).reshape(unique_data_type.count(1))
param_float_feature_indices = np.argwhere(np.array(unique_data_type)==2).reshape(unique_data_type.count(2))

print '%d bool, %d int and %d float' % (unique_data_type.count(0), 
                                        unique_data_type.count(1), 
                                        unique_data_type.count(2))

83 bool, 113 int and 110 float


In [5]:
#functions def
#for train&test dataset preprocessing

from sklearn.preprocessing import scale
from sklearn.preprocessing import maxabs_scale
from sklearn.preprocessing import robust_scale

#preprocess alg

def get_range_bool(train_data_bool):
    list_range_bool = []
    for i in range(train_data_bool.shape[1]):
        values = np.unique(train_data_bool[:,i])
        vzero = np.min(values)
        vone = np.max(values)
        list_range_bool.append([vzero,vone])
    return np.transpose(np.array(list_range_bool))

def get_range_int (train_data_int):
    #check int value range    
    list_range_int = []
    for i in range(train_data_int.shape[1]):
        list_range_int.append([np.min(train_data_int[:,i]),np.max(train_data_int[:,i])])
    return np.transpose(np.array(list_range_int))

def scale_float(float_features_ori):
    return maxabs_scale(float_features_ori, axis=0)

def scale_bool (bool_features_ori, bool_feature_range):
    bool_features = np.asarray(bool_features_ori)
    for i in range(bool_features.shape[0]):
        for j in range(bool_features.shape[1]):
            if bool_features[i,j]==bool_feature_range[0,j]:
                bool_features[i,j] = 0
            else:
                bool_features[i,j] = 1
    ##old version
    #bool_features = bool_features_ori - np.min(bool_features_ori, axis=0).reshape(1, bool_features_ori.shape[1])
    #bool_features = bool_features / np.max(bool_features, axis=0).reshape(1,bool_features.shape[1])
    return bool_features

def scale_int (int_features_ori, int_feature_range):
    int_features = np.array(int_features_ori)
    for i in range(int_features.shape[1]):
        pos_neg_feature = -2 #pos+neg
        if (int_feature_range[0,i] <0) & (int_feature_range[1, i] <=0):
            pos_neg_feature = -1
        if (int_feature_range[0,i] <0 ) & (int_feature_range[1, i] >0):
            pos_neg_feature = 0
        if (int_feature_range[0,i] >=0) & (int_feature_range[1, i] >0):
            pos_neg_feature = 1

        #scale
        for j in range(int_features.shape[0]):
            if int_features[j,i] < int_feature_range[0,i]:
                int_features[j,i] = int_feature_range[0,i];
            if int_features[j,i] > int_feature_range[1,i]:
                int_features[j,i] = int_feature_range[1,i];
            if pos_neg_feature == -1:
                int_features[j,i] = int_features[j,i]/-int_feature_range[0,i]
                if int_features[j,i] < -1:
                    int_features[j,i] = -1.0
            if pos_neg_feature == 0:
                if int_features[j,i] < 0:
                    int_features[j,i] = int_features[j,i]/-int_feature_range[0,i]
                    if int_features[j,i] < -1:
                        int_features[j,i] = -1.0
                else:
                    int_features[j,i] = int_features[j,i]/int_feature_range[1,i]
                    if int_features[j,i] > 1:
                        int_features[j,i] = 1.0
            if pos_neg_feature == 1:
                int_features[j,i] = int_features[j,i]/int_feature_range[1,i]
                if int_features[j,i] > 1:
                    int_features[j,i] = 1.0
    return int_features

#debug alg
def printarray(data):
    print data.shape
    print data
    #print np.unique(data)
    #print [np.sum(data==i) for i in np.unique(data)]
    #print np.unique(scale(test_data))
    #print np.unique(maxabs_scale(test_data))
    #print np.unique(robust_scale(test_data))
    #print np.sum(test_data==np.unique(test_data)[0])

In [6]:
#scale data based on type


do_type_part = True

if do_type_part:
    train_data_bool = train_data[:,param_bool_feature_indices]
    train_data_int = train_data[:, param_int_feature_indices]
    train_data_float = train_data[:, param_float_feature_indices]
    
    #test data preprocessing
    test_data_bool = test_data[:,param_bool_feature_indices]
    test_data_int = test_data[:, param_int_feature_indices]
    test_data_float = test_data[:, param_float_feature_indices]
    
    ##Generate MODEL PARAM
    param_bool_feature_range = get_range_bool(train_data_bool)
    #scale bool feature
    train_data_bool_scale = scale_bool(train_data_bool, param_bool_feature_range)
    test_data_bool_scale = scale_bool(test_data_bool, param_bool_feature_range)
    
    #Generate MODEL PARAM
    param_int_feature_range = get_range_int(train_data_int)
    #scale int feature:
    train_data_int_scale = scale_int(train_data_int, param_int_feature_range)
    test_data_int_scale = scale_int(test_data_int, param_int_feature_range)
    
    print train_data_int_scale
    
    #scale float feaure:
    data_float_scale = scale_float(np.row_stack((train_data_float, test_data_float)))
    train_data_float_scale = data_float_scale[:train_data_float.shape[0], :]
    test_data_float_scale = data_float_scale[train_data_float.shape[0] : , :]
    
    #printarray(train_data_float_scale[:,0])
    train_data_scale = np.column_stack((train_data_bool_scale, train_data_int_scale, train_data_float_scale))
    test_data_scale = np.column_stack((test_data_bool_scale, test_data_int_scale, test_data_float_scale))
    
    #train_data_scale = np.column_stack((train_data_bool, train_data_int, train_data_float))
    #test_data_scale = np.column_stack((test_data_bool, test_data_int, test_data_float))
    
    print train_data_scale.shape
    print test_data_scale.shape
    
print "Done type analysis and scale!"

[[ 0.00840336  0.21904762  0.         ...,  0.          0.          0.        ]
 [ 0.00840336  0.32380952  0.         ...,  0.          0.          0.        ]
 [ 0.00840336  0.21904762  0.         ...,  0.          0.          0.        ]
 ..., 
 [ 0.00840336  0.21904762  0.         ...,  0.          0.          0.        ]
 [ 0.00840336  0.23809524  0.         ...,  0.          0.          0.        ]
 [ 0.00840336  0.43809524  0.         ...,  0.          0.          0.        ]]
(76020, 306)
(75818, 306)
Done type analysis and scale!


In [10]:
cols = df_train.columns
#print cols[1:cols.shape[0]-1]
#print df_train.loc[:,cols[1:cols.shape[0]-1]]
df_train.loc[:,cols[1:cols.shape[0]-1]] = train_data_scale
df_test.loc[:,cols[1:cols.shape[0]-1]] = test_data_scale

In [12]:
df_train.to_csv('./data/train_data_scale.csv', index=False)
df_test.to_csv('./data/test_data_scale.csv', index=False)
print df_train.shape
print df_test.shape

(76020, 308)
(75818, 307)


#save scaled train/test data

#save param_* 
import csv
with open('data_processing_param.csv', 'w') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(param_selected_indices)
    f_csv.writerow(param_bool_feature_indices)
    f_csv.writerow(param_int_feature_indices)
    f_csv.writerow(param_float_feature_indices)
    f_csv.writerows(param_bool_feature_range)
    f_csv.writerows(param_int_feature_range)
    
    
#save header + scaled data
headers = np.ndarray((param_selected_indices.shape[0]+2),dtype=feature_names.dtype)
headers[1:param_selected_indices.shape[0]+1] = feature_names[param_selected_indices+1]
headers[0] = feature_names[0]
headers[param_selected_indices.shape[0]+1] = feature_names[feature_names.shape[0]-1]
data_scale_id_target = np.column_stack((train_id, train_data_scale, train_target))
print headers.shape
print data_scale_id_target.shape
with open('train_data_scale.csv', 'w') as f:
    f_csv = csv.writer(f)
    f_csv.writerows(data_scale_id_target)
print 'Done saving scaled training data'

#test data read/select/scale

#load params from taining data analysis
import csv
with open('./data_processing_param.csv') as f:
    f_csv = csv.reader(f)
    params = []
    for row in f_csv:
        params.append(np.array(row).astype(float))
param_selected_indices = params[0].astype(int)
param_bool_feature_indices = params[1].astype(int)
param_int_feature_indices = params[2].astype(int)
param_float_feature_indices = params[3].astype(int)
param_bool_feature_range = np.asarray(np.row_stack((params[4],params[5])))
param_int_feature_range = np.asarray(np.row_stack((params[6],params[7])))

#select features
test_data = test_data_ori[:,param_selected_indices.astype(int)]
test_data_bool = test_data[:,param_bool_feature_indices]
test_data_int = test_data[:,param_int_feature_indices]
test_data_float = test_data[:,param_float_feature_indices]

#scale bool features
test_data_bool_scale = scale_bool(test_data_bool, param_bool_feature_range)
test_data_int_scale = scale_int(test_data_int, param_int_feature_range)
test_data_float_scale = scale_float(test_data_float)

test_data_scale = np.column_stack((test_data_bool_scale, test_data_int_scale, test_data_float_scale))

#printarray(test_data_scale)
print 'Done scale test data'

In [13]:
#training start load preprocessed data
import pandas as pd
import numpy as np

df_train_scale = pd.read_csv('./data/train_data_scale.csv')
df_test_scale = pd.read_csv('./data/test_data_scale.csv')



#df_train_scale.replace(-99999, 2)
#df_test_scale.replace(-99999, 2)

train_data_target = df_train_scale['TARGET'].values
test_data_id = df_test_scale['ID'].values

train_data_scale = df_train_scale.drop(['ID','TARGET'], axis=1).values
test_data_scale = df_test_scale.drop(['ID'], axis=1).values

print df_train_scale.shape
print df_train_scale.drop(['ID', 'TARGET'], axis=1).shape

print train_data_scale.shape
print test_data_scale.shape


(76020, 308)
(76020, 306)
(76020, 306)
(75818, 306)


In [15]:
train_data_scale[:,-1]

array([ 0.00135684,  0.00170545,  0.00233034, ...,  0.00256202,
        0.00291676,  0.00405999])

In [16]:
#Boosting 
#ref: https://www.kaggle.com/cast42/santander-customer-satisfaction/xgboost-with-early-stopping/code

from sklearn import cross_validation as cv

print train_data_target.shape

# train_data_scale_n = np.row_stack((train_data_scale, train_data_scale[(train_data_target==1),:]))
# train_data_target_n = np.hstack((train_data_target, train_data_target[train_data_target==1]))

train_data_y = train_data_target.repeat(2).reshape([train_data_target.shape[0],2])
print train_data_y.shape
for i in xrange(train_data_y.shape[0]):
    train_data_y[i,1] = 1-train_data_y[i,0]

#x_train, x_valid, y_train, y_valid = \
#cv.train_test_split(train_data_scale, train_data_target, test_size=0.3, stratify=train_data_target)

x_train, x_valid, y_train, y_valid = \
cv.train_test_split(train_data_scale, train_data_y, test_size=0.3, stratify=train_data_target)

print x_train.shape
print x_valid.shape
print y_train.shape
print y_valid.shape


(76020,)
(76020, 2)
(53214, 306)
(22806, 306)
(53214, 2)
(22806, 2)


In [18]:
print np.sum(y_train[:,1]==0)
print np.sum(y_train[:,1]==1)
print np.sum(y_valid[:,1]==0)
print np.sum(y_valid[:,1]==1)
x_train = x_train.astype(np.float32)
y_train = y_train.astype(np.float32)

x_valid = x_valid.astype(np.float32)
y_valid = y_valid.astype(np.float32)

print x_train.shape
print y_train.shape

2106
51108
902
21904
(53214, 306)
(53214, 2)
[[ 0.          0.          1.         ...,  0.          0.          0.00116276]
 [ 0.          0.          0.         ...,  0.          0.          0.00142706]
 [ 0.          0.          1.         ...,  0.          0.          0.00673883]
 ..., 
 [ 0.          0.          1.         ...,  0.          0.          0.00146538]
 [ 0.          0.          1.         ...,  0.          0.          0.00357552]
 [ 0.          0.          1.         ...,  0.          0.          0.00243501]]


In [58]:
import tensorflow as tf
from sklearn.metrics import roc_auc_score

def weight_variable(shape):
  initial = tf.truncated_normal(shape, -np.sqrt(6.0/(np.sum(shape))), np.sqrt(6.0/(np.sum(shape))))
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.0, shape=shape)
  return tf.Variable(initial)

network_layer = 2
network_shape = [306, 150, 2]

x_in = tf.placeholder(tf.float32, shape = [None, network_shape[0]])
y_out = tf.placeholder(tf.float32, shape = [None, network_shape[network_layer]])

#W_mlp1 = tf.Variable(tf.random_uniform([306, 500], -1.0, 1.0))
#b_mlp1 = tf.Variable(tf.zeros([500]))

#W_mlp2 = tf.Variable(tf.random_uniform())
#b_mlp2 = 

W_mlp1 = weight_variable(network_shape[0:2])
b_mlp1 = bias_variable([network_shape[1]])

#W_mlp2 = weight_variable([500, 100])
#b_mlp2 = bias_variable([100])

W_mlp3 = weight_variable(network_shape[1:3])
b_mlp3 = bias_variable([network_shape[network_layer]])

sess = tf.Session()
sess.run(tf.initialize_all_variables())

# print sess.run(W)
# print sess.run(b)

out_mlp1 = tf.nn.tanh(tf.matmul(x_in, W_mlp1)+b_mlp1)
#out_mlp2 = tf.nn.tanh(tf.matmul(out_mlp1, W_mlp2)+b_mlp2)

y_pred = tf.nn.softmax(tf.matmul(out_mlp1, W_mlp3)+b_mlp3)

# y_pred_train = sess.run(out_mlp1, feed_dict={x_in:x_train[0:2,...]})

# print y_pred_train

# y_pred_train = sess.run(y_pred, feed_dict={x_in:x_train[0:2,...]})

# print y_pred_train

# print np.sum(y_pred_train[:,0]==0)
# print np.sum(y_pred_train[:,0]==1)



cross_entropy = -tf.reduce_sum(y_out*tf.log(y_pred))

train_step = tf.train.GradientDescentOptimizer(0.005).minimize(cross_entropy)

batchsize = 50
batchnum = int(x_train.shape[0]/batchsize)
epoch = 10
for i in xrange(epoch):
    #print 'epoch %d, batch %d/%d' % (i, j, batchnum)
    print 'epoch %d' % (i)
    for j in xrange(batchnum):
        
        x_batch = x_train[j*batchsize:(j+1)*batchsize,:]
        y_batch = y_train[j*batchsize:(j+1)*batchsize,:]
        sess.run(train_step, feed_dict={x_in: x_batch, y_out: y_batch})
#     print sess.run(W)
#     print sess.run(b)
    y_pred_train = sess.run(y_pred, feed_dict={x_in: x_train})
    y_pred_valid = sess.run(y_pred, feed_dict={x_in: x_valid})
    print "Train set ROC: %f, Valid set ROC: %f" % (roc_auc_score(y_train[:,0], y_pred_train[:,0]), \
                                                   roc_auc_score(y_valid[:,0], y_pred_valid[:,0]))

epoch 0
Train set ROC: 0.720097, Valid set ROC: 0.718136
epoch 1
Train set ROC: 0.744322, Valid set ROC: 0.737277
epoch 2
Train set ROC: 0.757180, Valid set ROC: 0.746223
epoch 3
Train set ROC: 0.762470, Valid set ROC: 0.751574
epoch 4
Train set ROC: 0.765548, Valid set ROC: 0.755340
epoch 5
Train set ROC: 0.767423, Valid set ROC: 0.757342
epoch 6
Train set ROC: 0.769257, Valid set ROC: 0.758795
epoch 7
Train set ROC: 0.770660, Valid set ROC: 0.759905
epoch 8
Train set ROC: 0.771852, Valid set ROC: 0.760737
epoch 9
Train set ROC: 0.772729, Valid set ROC: 0.761482


In [ ]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score

# ##[166]	validation_0-auc:0.883586	validation_1-auc:0.833347
# ##('Overall AUC:', 0.86816240395414446)
# ##test score: 0.830436
# clf = xgb.XGBClassifier(missing=9999999999,
#                 max_depth = 8,
#                 n_estimators=1000,
#                 learning_rate=0.05, 
#                 nthread=4,
#                 subsample=0.8,
#                 colsample_bytree=0.5,
#                 min_child_weight = 8,
#                 seed=4242)

# #('Overall AUC:', 0.87137364469597534)
# #Stopping. Best iteration:
# #[150]	validation_0-auc:0.887866	validation_1-auc:0.833577
# clf = xgb.XGBClassifier(missing=9999999999,
#                 max_depth = 9,
#                 n_estimators=1000,
#                 learning_rate=0.05, 
#                 nthread=4,
#                 subsample=0.8,
#                 colsample_bytree=0.5,
#                 min_child_weight = 8,
#                 seed=4242)

# ##('Overall AUC:', 0.8721795864254609)
# ##Stopping. Best iteration:
# ##[131]	validation_0-auc:0.889038	validation_1-auc:0.833293
# clf = xgb.XGBClassifier(missing=9999999999,
#                 max_depth = 10,
#                 n_estimators=1000,
#                 learning_rate=0.05, 
#                 nthread=4,
#                 subsample=0.8,
#                 colsample_bytree=0.5,
#                 min_child_weight = 8,
#                 seed=4242)


##('Overall AUC:', 0.88376225142894027)
##[203]	validation_0-auc:0.912779	validation_1-auc:0.832430
##Stopping. Best iteration:
##[153]	validation_0-auc:0.905169	validation_1-auc:0.833714
###double class 1 sample
##('Overall AUC:', 0.9541665781805323)
##[999]	validation_0-auc:0.972681	validation_1-auc:0.907279
# clf = xgb.XGBClassifier(missing=9999999999,
#                 max_depth = 10,
#                 n_estimators=1000,
#                 learning_rate=0.05, 
#                 nthread=4,
#                 subsample=0.8,
#                 colsample_bytree=0.5,
#                 min_child_weight = 5,
#                 seed=4242)

##('Overall AUC:', 0.89180781297900902)
##Stopping. Best iteration:
##[166]	validation_0-auc:0.916351	validation_1-auc:0.835339

clf = xgb.XGBClassifier(missing=9999999999,
                max_depth = 12,
                n_estimators=1000,
                learning_rate=0.05, 
                nthread=4,
                subsample=0.8,
                colsample_bytree=0.5,
                min_child_weight = 8,
                seed=1030)

clf.fit(x_train, y_train, early_stopping_rounds=70, eval_metric="auc",
        eval_set=[(x_train, y_train), (x_test, y_test)])
        
print('Overall AUC:', roc_auc_score(train_data_target, clf.predict_proba(train_data_scale, ntree_limit=clf.best_iteration)[:,1]))

In [ ]:
#test
y_pred = clf.predict_proba(test_data_scale, ntree_limit=clf.best_iteration)
#print y_pred
submission = pd.DataFrame({"ID":test_data_id, "TARGET":y_pred[:,1]})
submission.to_csv("submission_nofloat_noint.csv", index=False)

y_pred_class = clf.predict(test_data_scale, ntree_limit=clf.best_iteration)
print np.unique(y_pred_class)
print np.sum(y_pred_class == 0)
print np.sum(y_pred_class == 1)
#submission_class = submission = pd.DataFrame({"ID":test_data_id, "TARGET":y_pred_class})
#submission_class.to_csv('submission_class.csv', index=False)

In [ ]:
#Find the wrong answers
train_pred_target = clf.predict

In [ ]:
#pca analyses
#ref:https://www.kaggle.com/tuomastik/santander-customer-satisfaction/pca-visualization/code

#data normalization

from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA

do_l2_norm = True

#train_data normalization
if do_l2_norm:
    train_data_norm = normalize(train_data_scale, axis=0)
else:
    train_data_norm = train_data_scale
#print np.sum(train_data_norm[:,:2], axis=0)
pca = PCA(n_components=2)
train_data_pca = pca.fit_transform(train_data_norm)

print "Done PCA analysis with components = 2"


#plot

%matplotlib inline
from matplotlib import pyplot as plt

classes = np.sort(np.unique(train_target))
labels = ["Satisfied customer", "Unsatisfied customer"]
# Visualize
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(1, 1, 1)
colors = [(0.0, 0.63, 0.69), 'black']
markers = ["o", "D"]
print type(zip(classes, markers, colors, train_target))
print zip(classes, markers, colors, train_target)
for class_ix, marker, color, label in zip(
        classes, markers, colors, labels):
    print 'll: '+ str(class_ix) + str(marker) + str(color) + str(label)
    ax.scatter(train_data_pca[train_target==class_ix, 0],
               train_data_pca[train_target==class_ix, 1],
               c=color, marker=marker, edgecolor='whitesmoke', 
               linewidth='1', alpha=0.9, label=label)
    ax.legend(loc='best')

plt.title(
    "Scatter plot of the training data examples projected on the "
    "2 first principal components")
plt.xlabel("Principal axis 1 - Explains %.1f %% of the variance" % (
    pca.explained_variance_ratio_[0] * 100.0))
plt.ylabel("Principal axis 2 - Explains %.1f %% of the variance" % (
    pca.explained_variance_ratio_[1] * 100.0))
plt.show()
#plt.savefig("./pca.pdf", format='pdf')
#plt.savefig("./pca.png", format='png')


#for dev, use 1 batch
batch_size = 100
train_data_b = train_data[:100]

print nsample
print nfeature

for class_ix, marker, color, label in zip(
        classes, markers, colors, train_target):
    ax.scatter(train_data_pca[np.where(train_target == class_ix), 0],
               train_data_pca[np.where(train_target == class_ix), 1],
               marker=marker, color=color, edgecolor='whitesmoke',
               linewidth='1', alpha=0.9, label=label)
    ax.legend(loc='best')
plt.title(
    "Scatter plot of the training data examples projected on the "
    "2 first principal components")
plt.xlabel("Principal axis 1 - Explains %.1f %% of the variance" % (
    pca.explained_variance_ratio_[0] * 100.0))
plt.ylabel("Principal axis 2 - Explains %.1f %% of the variance" % (
    pca.explained_variance_ratio_[1] * 100.0))
plt.show()
plt.savefig("pca.pdf", format='pdf')
plt.savefig("pca.png", format='png')